In [1]:
import csv
import os
import pickle
import re
from pathlib import Path
import pandas as pd

## Join data betweeen experiment settings and axial-force meta data

In [2]:
exps = pd.read_excel("../T01_explore/S04/S04_residual_stress_imputed_20251222_105458.xlsx")
metas = pd.read_excel("S10_af_meta.xlsx")

In [3]:
exps

,idx_excel_post,section,sample_no,location,R,W,D,sigma_x_post,FWHM_post,idx_excel_pre,sigma_x_pre,FWHM_pre,diff_sigma_x
0,4,AA5052,1,1,1400,60,10,13.0,2.55,2,-15.0,2.50,28
1,25,AA5052,2,1,1400,60,15,16.0,2.51,9,2.0,2.47,14
2,46,AA5052,3,1,1400,60,20,19.0,2.47,16,9.0,2.48,10
3,67,AA5052,4,1,1400,70,10,20.0,2.45,23,10.0,2.48,10
4,88,AA5052,5,1,1400,70,15,6.0,2.47,30,0.0,2.49,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1129,1050,Center,50,7,1600,70,15,2.0,2.45,0,0.0,0.00,2
1130,1071,Center,51,7,1600,70,20,2.0,2.45,0,0.0,0.00,2
1131,1092,Center,52,7,1600,80,10,5.0,2.54,0,0.0,0.00,5
1132,1113,Center,53,7,1600,80,15,1.0,2.41,0,0.0,0.00,1


In [4]:
# Check within group. The quantities R, D, W should be the same within each sample_no


def check_within_group(df, qty):
    unique_vals = df[qty].unique()
    if len(unique_vals) > 1:
        print(f"Different {qty} values found in sample_no:", df.name, unique_vals)


_ = exps.groupby("sample_no").apply(
    lambda df: check_within_group(df, "R"), include_groups=False
)
_ = exps.groupby("sample_no").apply(
    lambda df: check_within_group(df, "D"), include_groups=False
)
_ = exps.groupby("sample_no").apply(
    lambda df: check_within_group(df, "W"), include_groups=False
)

In [5]:
# Assuming no differences found above, we can now merge the dataframes
settings = exps.groupby("sample_no").first().reset_index()[["sample_no", "R", "D", "W"]]

# Save settings for future reference
settings.to_excel("S11_settings.xlsx", index=False)

In [6]:
dfm = metas.merge(settings, on="sample_no", how="left")
dfm

,sample_no,DynoWare,Path,Filename,Config ID,Setup ID,Manipulated,Filename 1,Filename 2,Date,...,Delay time [s],Cycle time [s],Cycles,Samples per channel,Cycle interval,Cycle No,filename,R,D,W
0,1,Version 3.2.2.0,D:\Parn (paper ML Weld)\Post welding\,Sample 1.dwd,Sample 1.cfg,0,0,NaN,NaN,"Wednesday, January 01, 2025",...,0,0,1,18001,0,1,Sample 1.csv,1400,10,60
1,2,Version 3.2.2.0,D:\Parn (paper ML Weld)\Post welding\,Sample 2.dwd,Sample 2.cfg,0,0,NaN,NaN,"Wednesday, January 01, 2025",...,0,0,1,18001,0,1,Sample 2.csv,1400,15,60
2,3,Version 3.2.2.0,D:\Parn (paper ML Weld)\Post welding\,Sample 3.dwd,Sample 3.cfg,0,0,NaN,NaN,"Wednesday, January 01, 2025",...,0,0,1,18001,0,1,Sample 3.csv,1400,20,60
3,4,Version 3.2.2.0,D:\Parn (paper ML Weld)\Post welding\,Sample 4.dwd,Sample 4.cfg,0,0,NaN,NaN,"Wednesday, January 01, 2025",...,0,0,1,18001,0,1,Sample 4.csv,1400,10,70
4,5,Version 3.2.2.0,D:\Parn (paper ML Weld)\Post welding\,Sample 5.dwd,Sample 5.cfg,0,0,NaN,NaN,"Wednesday, January 01, 2025",...,0,0,1,18001,0,1,Sample 5.csv,1400,15,70
5,6,Version 3.2.2.0,D:\Parn (paper ML Weld)\Post welding\,Sample 6.dwd,Sample 6.cfg,0,0,NaN,NaN,"Wednesday, January 01, 2025",...,0,0,1,18001,0,1,Sample 6.csv,1400,20,70
6,7,Version 3.2.2.0,D:\Parn (paper ML Weld)\Post welding\,Sample 7.dwd,Sample 7.cfg,0,0,NaN,NaN,"Wednesday, January 01, 2025",...,0,0,1,18001,0,1,Sample 7.csv,1400,10,80
7,8,Version 3.2.2.0,D:\Parn (paper ML Weld)\Post welding\,Sample 8.dwd,Sample 8.cfg,0,0,NaN,NaN,"Wednesday, January 01, 2025",...,0,0,1,18001,0,1,Sample 8.csv,1400,15,80
8,9,Version 3.2.2.0,D:\Parn (paper ML Weld)\Post welding\,Sample 9.dwd,Sample 9.cfg,0,0,NaN,NaN,"Wednesday, January 01, 2025",...,0,0,1,18001,0,1,Sample 9.csv,1400,20,80
9,10,Version 3.2.2.0,D:\Parn (paper ML Weld)\Post welding\,Sample 10.dwd,Sample 10.cfg,0,0,NaN,NaN,"Wednesday, January 01, 2025",...,0,0,1,18001,0,1,Sample 10.csv,1500,10,60


In [7]:
dfm.to_excel("S11_af_meta.xlsx", index=False)